# 06 Generate Validation Data with Background
#### Author: Subhojeet Pramanik
#### Cognibit Solutions LLP

Add Background noise and save validation data.

In [1]:
import tensorflow as tf
import os
import sys
import numpy as np

sys.path.append("../libs")
from classification import label_wav
from classification import input_data
from classification import models

/home/metal_geek/anaconda3/lib/python3.6/importlib/_bootstrap.py:205: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)
/home/metal_geek/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


### Flags

In [2]:
flags=tf.app.flags
#Important Directories
flags.DEFINE_string('data_dir','../data/raw','Train Data Folder')
flags.DEFINE_string('summaries_dir','../summaries','Summaries Folder')
flags.DEFINE_string('train_dir','../logs&checkpoint','Directory to write event logs and checkpoint')
flags.DEFINE_string('models_dir','../models','Models Folder')
flags.DEFINE_string('wanted_words','yes,no,up,down,left,right,on,off,stop,go','Wanted Words')
flags.DEFINE_float('validation_percentage',20,'Validation Percentage')
flags.DEFINE_float('testing_percentage',0,'Testing Percentage')
flags.DEFINE_integer('sample_rate',16000,'Sample Rate')
flags.DEFINE_integer('clip_duration_ms',1000,'Clip Duration in ms')
flags.DEFINE_float('window_size_ms',40,'How long each spectogram timeslice is')
flags.DEFINE_float('window_stride_ms',20.0,'How far to move in time between frequency windows.')
flags.DEFINE_integer('dct_coefficient_count',40,'How many bins to use for the MFCC fingerprint')
flags.DEFINE_float('time_shift_ms',200.0,'Range to randomly shift the training audio by in time.')
FLAGS=flags.FLAGS

### Variables

In [3]:
silence_percentage=10.0
unknown_percentage=10
background_frequency=0.8
background_volume=0.8
save_folder='../data/train'

### Prepare the Audio Processor

In [4]:
train_dir=os.path.join(FLAGS.data_dir,'train','audio')
model_settings = models.prepare_model_settings(
      len(input_data.prepare_words_list(FLAGS.wanted_words.split(','))),
      FLAGS.sample_rate, FLAGS.clip_duration_ms, FLAGS.window_size_ms,
      FLAGS.window_stride_ms, FLAGS.dct_coefficient_count)
audio_processor = input_data.AudioProcessor(
      train_dir, silence_percentage, unknown_percentage,
      FLAGS.wanted_words.split(','), FLAGS.validation_percentage,
      FLAGS.testing_percentage, model_settings)

../data/raw/train/audio/*/*.wav
../data/raw/train/audio/_background_noise_/pink_noise.wav
../data/raw/train/audio/_background_noise_/doing_the_dishes.wav
../data/raw/train/audio/_background_noise_/white_noise.wav
../data/raw/train/audio/_background_noise_/dude_miaowing.wav
../data/raw/train/audio/_background_noise_/running_tap.wav
../data/raw/train/audio/_background_noise_/exercise_bike.wav
Tensor("AudioSpectrogram:0", shape=(?, 49, 513), dtype=float32)


### Get the Audio Files as Raw and Truth Labels

In [5]:
labels_list=input_data.prepare_words_list(FLAGS.wanted_words.split(','))

In [7]:
with tf.Session() as sess:
    data,one_hot=audio_processor.get_data_raw(-1,0,model_settings,background_frequency,background_volume,FLAGS.time_shift_ms,
                             'training',sess)
one_hot=one_hot.argmax(axis=1)
truth_labels=[]
for i in range(0,one_hot.shape[0]):
    truth_labels.append(labels_list[one_hot[i]])
truth_labels=np.array(truth_labels)

## Save the Audio Files

In [ ]:
for i in range(truth_labels.shape[0]):
    save_file=os.path.join(save_folder,'test','audio','%s-%d.wav'%(truth_labels[i],i))
    input_data.save_wav_file(save_file,data[i],FLAGS.sample_rate)